# Sequential Monte Carlo Inference of Evolutionary Parameters

In [15]:
# Libraries
using EvolutionaryModels 
using KissABC
using Distributions
using ClusterManagers
using DataFrames
using Setfield
using StatsBase
using CSV
using Distances
using Plots

In [2]:
######### Prepare data to be fitted
data = CSV.read("../data/Top2_Population_Counts.csv",DataFrame;drop=[1])
Tdata = EvolutionaryModels.TargetData(data)
# Timepoints sampled
TimePoint = Tdata[1]
# Target data to fit
tdata = Tdata[2]
# Number of barcodes
barcodes = Tdata[3]
# Initial condition of each barcoded Lineage
n0 = Tdata[4]
Tdata

(String3["T0", "T1", "T2", "T3", "T5", "T6", "T8", "T9", "T10", "T13", "T15", "T19"], [16768.5043743925 13510.6235245105 2.7297208721011e6; 100760.465692346 65129.8819255222 2.01810965238213e6; … ; 4.70701824477294e6 0.0 42981.7552270609; 1.3927694087771e7 153.522272547381 37152.3899564663], 3, [16768.5043743925, 13510.6235245105, 2.7297208721011e6])

In [16]:
######### TimeSeries Parameters
parameters = CSV.read("../data/Complete_MasterSizes.csv",DataFrame;drop=[1])
Parametres = EvolutionaryModels.Params(parameters)
# Times between culutres
TimeCultures = Parametres[1]
# Cells transferred
Ntransferes = Parametres[2]
Passes = Parametres[3]
# Global Parameters
NPasses = Parametres[4]
Parametres

([0, 18, 14, 11, 18, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 13, 14, 14], [500000, 500000, 500000, 500000, 500000, 500000, 500000, 400000, 250000, 250000, 250000, 125000, 250000, 250000, 250000, 250000, 250000, 250000, 250000, 250000], String3["T0", "T1", "T2", "T3", "T4", "T5", "T6", "T7", "T8", "T9", "T10", "T11", "T12", "T13", "T14", "T15", "T16", "T17", "T18", "T19"], 20)

In [4]:
# Define the prior
prior=Factored(Uniform(0,1),# s_RG0
              Uniform(0,1),  #s_RG1
              Uniform(0,1))  #S_Remainder

Factored{3}(p=(Uniform{Float64}(a=0.0, b=1.0), Uniform{Float64}(a=0.0, b=1.0), Uniform{Float64}(a=0.0, b=1.0)))

In [17]:
NPasses

20

In [5]:
@time ressmc = smc(prior, EvolutionaryModels.costtest, nparticles=20, epstol=10,verbose=true,parallel=true)

LoadError: TaskFailedException

[91m    nested task error: [39mUndefVarError: NPasses not defined
    Stacktrace:
     [1] [0m[1msimtest[22m[0m[1m([22m::[0mTuple[90m{Float64, Float64, Float64}[39m[0m[1m)[22m
    [90m   @ [39m[35mEvolutionaryModels[39m [90m~/Documents/Workstation/PechuanLab/EvolutionaryModels/src/[39m[90m[4mCurtisBarcodeModel.jl:527[24m[39m
     [2] [0m[1mcosttest[22m[0m[1m([22m::[0mTuple[90m{Float64, Float64, Float64}[39m[0m[1m)[22m
    [90m   @ [39m[35mEvolutionaryModels[39m [90m~/Documents/Workstation/PechuanLab/EvolutionaryModels/src/[39m[90m[4mInferenceHelpers.jl:30[24m[39m
     [3] [0m[1m(::KissABC.var"#12#25"{Int64, Factored{3}, typeof(EvolutionaryModels.costtest)})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @ [39m[36mKissABC[39m [90m./[39m[90m[4mthreadingconstructs.jl:178[24m[39m

In [18]:
@time EvolutionaryModels.simtest((0.25,0.2,0.1)) 

LoadError: UndefVarError: NPasses not defined

In [7]:
@time sim((0.25,0.2,0.1)) 

LoadError: UndefVarError: sim not defined

In [21]:
@time simtest((0.25,0.2,0.1),NPasses,barcodes,TimeCultures,Ntransferes)

  0.258656 seconds (160.72 k allocations: 61.357 MiB, 7.86% gc time, 24.49% compilation time)


,RG0,RG1,Remainder,TimePoint
,Float64,Float64,Float64,String3
1,16768.5,13510.6,2.72972e6,T0
2,2.80187e5,88967.5,2.99098e6,T1
3,1.37747e6,214727.0,1.80922e6,T2
4,3.17015e6,2.83655e5,7.98907e5,T3
5,1.22319e7,2.3512e5,30051.5,T5
6,1.61737e7,1.538e5,4934.19,T6
7,1.31392e7,32607.3,57.3312,T8
8,8.23852e6,10078.8,0.0,T9
9,8.29546e6,4684.77,0.0,T10


In [20]:
function simtest((s_RG0,s_RG1,S_Remainder),NPasses,barcodes,TimeCultures,Ntransferes) 
  # Fix
  # Selective coefficients     
  s_coef = [s_RG0,s_RG1,S_Remainder]
  # Things we will record
  BarCode_mat = zeros(Float64,NPasses,barcodes)
  # Initial Population
  poblacio = EvolutionaryModels.InitPop(barcodes,s_coef,n0)
  # Culture cycles
  for i in 1:NPasses
      TimeCulture = TimeCultures[i]
      Ntransfer = Ntransferes[i]
      poblacio =  EvolutionaryModels.CulturePasss(poblacio::Population,TimeCulture)
      N_vec =  EvolutionaryModels.unwrapper(poblacio,"N")
      BarCode_mat[i,:] = N_vec
      poblacio =  EvolutionaryModels.Transfer(poblacio::Population,Ntransfer::Int,barcodes::Int)
  end
  subset_BarcodeMat =  EvolutionaryModels.measurement_Sample(BarCode_mat,TimePoint,Passes,data)
  return subset_BarcodeMat
end

simtest (generic function with 1 method)

In [9]:
function sim((s_RG0,s_RG1,S_Remainder)) 

end


sim (generic function with 1 method)

In [10]:
  # Fix
  # Selective coefficients     
  s_coef = [0.25,0.2,0.1]
  # Things we will record
  BarCode_mat = zeros(Float64,NPasses,barcodes)
  # Initial Population
  poblacio = EvolutionaryModels.InitPop(barcodes,s_coef,n0)

Population(EvolutionaryModels.Lineages[EvolutionaryModels.Lineage(1, 0.25, 16768.5043743925), EvolutionaryModels.Lineage(2, 0.2, 13510.6235245105), EvolutionaryModels.Lineage(3, 0.1, 2.7297208721011e6)])

In [11]:
poblacio.clones[1].barcode

1

In [12]:
############### Top Types
abstract type Lineages end
abstract type Populations end

############### Lineage Types
# Lineage without mutation
mutable struct Lineage <: Lineages
    barcode::Int
    fitness::Float64
    N::Float64
end


In [13]:
barcodes

3